# Use NDWI/NDVI for Initial Classification

In [ ]:
import geopandas as gpd
import os
import rasterio
import rasterio.mask
from pathlib import Path
from rasterio.plot import show
from rasterio.crs import CRS
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import numpy as np
from tools.lookup import connect_site_and_planet_ids

# Parameters

Specify *exactly* one. The `site_name` or the `planet_id`. The former is given to the chip by the validation team. Because we are not selecting multiple planet scenes per chip and not selecting planet images that cover multiple chips (they are sufficiently spaced apart), this should be a 1 to 1 mapping.

In [ ]:
PLANET_ID = '20210903_150800_60_2458'
SITE_NAME = ''

# Connect IDs

In [ ]:
data = connect_site_and_planet_ids({'planet_id': PLANET_ID, 
                                    'site_name': SITE_NAME})
data

In [ ]:
PLANET_ID = data['planet_id']
SITE_NAME = data['site_name']

# Indices: NDWI and NDVI

Source: https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/toar/toar_planetscope.ipynb

## Metadata

In [ ]:
data_dir = Path(f'data/{PLANET_ID}/')
data_dir.mkdir(exist_ok=True, parents=True)

cropped_dir = Path(f'planet_images_cropped/{PLANET_ID}/')
cropped_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
metadata_xml = list(data_dir.glob('*metadata.xml'))[0]
metadata_xml

In [ ]:
original_path = list(data_dir.glob(f'{PLANET_ID}*.tif'))[0]

with rasterio.open(original_path) as ds:
    desc = list(enumerate(ds.descriptions, start=1))
desc

In [ ]:
with rasterio.open(cropped_dir / f'cropped_{PLANET_ID}.tif') as ds:
    image_c = ds.read().transpose([1, 2, 0]).astype(np.float32)
    p = ds.profile
    mask = ~ds.read_masks(1).astype(bool)

In [ ]:
from xml.dom import minidom

xmldoc = minidom.parse(str(metadata_xml))
nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

# XML parser refers to bands by numbers 1-4
coeffs = {}
for node in nodes:
    bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
    if bn in [str(n) for n in range(1, 9)]:
        i = int(bn)
        value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
        coeffs[i] = float(value)

coeffs

In [ ]:
## If you have 4 band imagery image you will have to N = 4
# N = 4
N = 8
coeffs_arr = np.array([coeffs[k] for k in range(1, N + 1)])
image_c_sr = image_c * coeffs_arr



## NDWI

In [ ]:
ndwi = (image_c_sr[..., 3] - image_c_sr[..., 7]) / (image_c_sr[..., 3] + image_c_sr[..., 7])

## If you have 4 band use below
# ndwi = (image_c_sr[..., 1] - image_c_sr[..., 3]) / (image_c_sr[..., 1] + image_c_sr[..., 3])

In [ ]:
plt.imshow(ndwi)
plt.colorbar()

In [ ]:
ndwi_t = (ndwi > -.6).astype(np.uint8)
plt.imshow(ndwi_t, interpolation='none')

In [ ]:
from scipy import signal

def majority_filter(arr, n_pixels=4):
    arr_maj = signal.convolve2d(arr, np.ones((3, 3)), boundary='symm', mode='same')
    arr_maj = (arr_maj > n_pixels).astype(np.uint8)
    return arr_maj

ndwi_t = majority_filter(ndwi_t)
plt.imshow(ndwi_t, interpolation='none')

In [ ]:
p_new = p.copy()
p_new.update({'count': 1, 'dtype': 'uint8', 'nodata': 255})
with rasterio.open(cropped_dir / f'ndwi_thresh_{PLANET_ID}.tif', 'w', **p_new) as ds:
    ds.write(ndwi_t, 1)

In [ ]:
p_new = p.copy()
p_new.update({'count': 1, 'dtype': 'float32', 'nodata': 255})
with rasterio.open(cropped_dir / f'ndwi_{PLANET_ID}.tif', 'w', **p_new) as ds:
    ds.write(ndwi.astype(np.float32), 1)

## NDVI

In [ ]:
ndvi = (image_c_sr[..., 7] - image_c_sr[..., 5]) / (image_c_sr[..., 7] + image_c_sr[..., 5])

plt.imshow(ndvi)
plt.colorbar()

In [ ]:
ndvi_t = (ndvi < 0.6).astype(np.uint8)


ndvi_t = majority_filter(ndvi_t)
plt.imshow(ndvi_t, interpolation='none')

In [ ]:
p_new = p.copy()
p_new.update({'count': 1, 'dtype': 'uint8', 'nodata': 255})
with rasterio.open(cropped_dir / f'ndvi_thresh_{PLANET_ID}.tif', 'w', **p_new) as ds:
    ds.write(ndvi_t, 1)

# Final combination

Selected based on scene and manual thresholds above. Do some boolean logic that makes sense for your scene.

In [ ]:
water = ndwi_t #| ndvi_t
plt.imshow(water, interpolation='none')

In [ ]:
p_new = p.copy()
p_new.update({'count': 1, 'dtype': 'uint8', 'nodata': 255})
class_t_final = np.zeros(mask.shape)

class_t_final[mask.astype(bool)] = 255
class_t_final[water.astype(bool)] = 1

with rasterio.open(cropped_dir / f'classification_{PLANET_ID}.tif', 'w', **p_new) as ds:
    ds.write(class_t_final, 1)